# Ventanas temporales de procesamiento - `window`, `countByWindow` y `countByValueAndWindow`

In [ ]:
import org.apache.spark._

In [ ]:
import org.apache.spark.streaming._

In [ ]:
val streamctx = new StreamingContext(sc, Seconds(15))

In [ ]:
val datos = streamctx.socketTextStream("localhost", 19000)

In [ ]:
// Obligatorio al trabajar con estados.
streamctx.checkpoint("/eoi/streaming/checkpoint")

In [ ]:
datos.print()

In [ ]:
case class Medicion(nodo:String, sensor:String, valor:String)

In [ ]:
val medidas = datos.flatMap(x => {
    val campos = x.split(";")
    try {
        List(Medicion(campos(0), campos(1), campos(2)))
    }
    catch {
        case e : Throwable => {
            // LLamar al sistema de notificación en caso de fallo en la entrada. 
        }
        List ()
    }
})

In [ ]:
// Medidas de dentro de la ventana (45 segundos con desplazamiento de 15 segundos)
val ventana = medidas.map(x => (x.sensor, x.valor)).window(Seconds(45), Seconds(15))

In [ ]:
ventana.print()

In [ ]:
// Número de medidas dentro de la ventana (45 segundos con desplazamiento de 15 segundos)
val contar_ventana = medidas.map(x => (x.sensor, x.valor)).countByWindow(Seconds(45), Seconds(15))

In [ ]:
contar_ventana.print()

In [ ]:
// Número de medidas dentro de la ventana (45 segundos con desplazamiento de 15 segundos)
val contar_ventana_valor = medidas.map(x => (x.sensor)).countByValueAndWindow(Seconds(45), Seconds(15))

In [ ]:
contar_ventana_valor.print()

In [ ]:
streamctx.start()

In [ ]:
streamctx.stop()

# Ventanas temporales de procesamiento - `reduceByWindow`

In [ ]:
val streamctx = new StreamingContext(sc, Seconds(15))

In [ ]:
val datos = streamctx.socketTextStream("localhost", 19000)

In [ ]:
// Obligatorio al trabajar con estados.
streamctx.checkpoint("/eoi/streaming/checkpoint")

In [ ]:
datos.print()

In [ ]:
case class Medicion(nodo:String, sensor:String, valor:String)

In [ ]:
val medidas = datos.flatMap(x => {
    val campos = x.split(";")
    try {
        List(Medicion(campos(0), campos(1), campos(2)))
    }
    catch {
        case e : Throwable => {
            // LLamar al sistema de notificación en caso de fallo en la entrada. 
        }
        List ()
    }
})

In [ ]:
// Medidas de dentro de la ventana (45 segundos con desplazamiento de 15 segundos)
val ventana = medidas.map(x => (x.nodo, x.sensor, x.valor)).window(Seconds(45), Seconds(15))

In [ ]:
ventana.print

In [ ]:
// Máximo de temperatura dentro de la ventana (45 segundos con desplazamiento de 15 segundos)
val reduce_ventana_1 = medidas.filter(x => x.sensor == "Temp").
map(x => (x.sensor, x.valor.toDouble)).
reduceByWindow((v1, v2) => if (v1._2 > v2._2) v1 else v2, Seconds(45), Seconds(15))

In [ ]:
reduce_ventana_1.print

In [ ]:
// Medidas producidas por cada nodo dentro de la ventana (45 segundos con desplazamiento de 15 segundos)
val reduce_ventana_2 = medidas.map(x => (x.nodo, 1)).reduceByWindow((v1,v2) => (v1._1, v1._2 + v2._2), 
                                                                    (w1, w2) => (w1._1, w1._2 - w2._2), 
                                                                    Seconds(45), Seconds(15))

In [ ]:
reduce_ventana_2.print

In [ ]:
streamctx.start()

In [ ]:
streamctx.stop()

# Ventanas temporales de procesamiento - `reduceByKeyAndWindow`

In [ ]:
val streamctx = new StreamingContext(sc, Seconds(15))

In [ ]:
val datos = streamctx.socketTextStream("localhost", 19000)

In [ ]:
// Obligatorio al trabajar con estados.
streamctx.checkpoint("/eoi/streaming/checkpoint")

In [ ]:
datos.print()

In [ ]:
case class Medicion(nodo:String, sensor:String, valor:String)

In [ ]:
val medidas = datos.flatMap(x => {
    val campos = x.split(";")
    try {
        List(Medicion(campos(0), campos(1), campos(2)))
    }
    catch {
        case e : Throwable => {
            // LLamar al sistema de notificación en caso de fallo en la entrada. 
        }
        List ()
    }
})

In [ ]:
// Medidas de dentro de la ventana (45 segundos con desplazamiento de 15 segundos)
val ventana = medidas.map(x => (x.nodo, x.sensor, x.valor)).window(Seconds(45), Seconds(15))

In [ ]:
ventana.print()

In [ ]:
// Número de medidas producidas de cada tipo dentro de la ventana (45 segundos con desplazamiento de 15 segundos)
val reduce_ventana_1 = medidas.filter(x => x.sensor == "Temp" || x.sensor == "Humd").
map(x => (x.sensor, 1)).
reduceByKeyAndWindow((v1:Int, v2:Int) => v1 + v2, Seconds(45), Seconds(15))

In [ ]:
reduce_ventana_1.print

In [ ]:
// Número de medidas producidas de cada tipo dentro de la ventana (45 segundos con desplazamiento de 15 segundos).
// Sólo incluir aquellas que aparecen más de una vez.
val reduce_ventana_2 = medidas.filter(x => x.sensor == "Temp" || x.sensor == "Humd").
map(x => (x.sensor, 1)).
reduceByKeyAndWindow((v1:Int, v2:Int) => v1 + v2, 
                (w1:Int, w2:Int) => w1 - w2, 
                Seconds(45), Seconds(15), 3, (x: (String,Int)) => x._2 != 0)

In [ ]:
reduce_ventana_2.print

In [ ]:
streamctx.start()

In [ ]:
streamctx.stop()

# Ventanas temporales de procesamiento - `groupByKeyAndWindow`

In [ ]:
val streamctx = new StreamingContext(sc, Seconds(15))

In [ ]:
val datos = streamctx.socketTextStream("localhost", 19000)

In [ ]:
// Obligatorio al trabajar con estados.
streamctx.checkpoint("/eoi/streaming/checkpoint")

In [ ]:
datos.print()

In [ ]:
case class Medicion(nodo:String, sensor:String, valor:String)

In [ ]:
val medidas = datos.flatMap(x => {
    val campos = x.split(";")
    try {
        List(Medicion(campos(0), campos(1), campos(2)))
    }
    catch {
        case e : Throwable => {
            // LLamar al sistema de notificación en caso de fallo en la entrada. 
        }
        List ()
    }
})

In [ ]:
// Medidas de dentro de la ventana (45 segundos con desplazamiento de 15 segundos)
val ventana = medidas.map(x => (x.nodo, x.sensor, x.valor)).window(Seconds(45), Seconds(15))

In [ ]:
ventana.print()

In [ ]:
// Medidas producidas por los botonoes dentro de la ventana (45 segundos con desplazamiento de 15 segundos)
val group_ventana = medidas.filter(x => x.sensor == "Boton").
map(x => (x.sensor, x.valor)).
groupByKeyAndWindow(Seconds(45), Seconds(15))

In [ ]:
group_ventana.print

In [ ]:
streamctx.start()

In [ ]:
streamctx.stop()